<a href="https://colab.research.google.com/github/Kelzo8/AI/blob/main/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# Load dataset
df = pd.read_csv("/content/Sleep_health_and_lifestyle_dataset.csv")

# Display dataset information
df.info()
df.head()

In [ ]:
# Helps visualize whether sleep duration impacts sleep quality.
df.plot(kind='scatter', x='Sleep Duration', y='Quality of Sleep', s=32, alpha=.8)
plt.gca().spines[['top', 'right']].set_visible(False)

# Shows how sleep duration is distributed among individuals.
plt.figure(figsize=(8, 5))
sns.histplot(df["Sleep Duration"], bins=20, kde=True, color="blue")
plt.title("Distribution of Sleep Duration")
plt.xlabel("Sleep Duration (hours)")
plt.ylabel("Count")
plt.show()

In [ ]:
# Helps see how stress affects sleep duration.
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x="Stress Level", y="Sleep Duration", palette="coolwarm")
plt.title("Stress Level vs. Sleep Duration")
plt.xlabel("Stress Level")
plt.ylabel("Sleep Duration (hours)")
plt.show()

In [ ]:
# Displays how sleep disorders vary between genders.
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="Gender", hue="Sleep Disorder", palette="viridis")
plt.title("Sleep Disorder Count by Gender")
plt.xlabel("Gender")
plt.ylabel("Count")
plt.legend(title="Sleep Disorder")
plt.show()

In [ ]:
# Drop Person ID since it's not a relevant feature
df = df.drop(columns=["Person ID"])

# Encode categorical variables
label_enc = LabelEncoder()
df["Gender"] = label_enc.fit_transform(df["Gender"])
df["BMI Category"] = label_enc.fit_transform(df["BMI Category"])
df["Occupation"] = label_enc.fit_transform(df["Occupation"])

# Define features and target variable
X = df.drop(columns=["Quality of Sleep"]).values
y = df["Quality of Sleep"].values

In [ ]:
# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')  # Regression output
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=1)

# Evaluate model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae}")